In [1]:
from pydrake.all import (MathematicalProgram, Solve, MonomialBasis,
                         DiagramBuilder, Evaluate, LogVectorOutput, Simulator,
                         SymbolicVectorSystem, Variable, ToLatex, Polynomial,
                         VectorSystem, eq, ge, le, Formula, Expression, Evaluate,
                         LeafSystem, AbstractValue,
                         )

from pydrake.all import (PiecewisePolynomial, ModelInstanceIndex,
    RotationMatrix, RigidTransform, Rgba, Box, Sphere, BaseField,
    Evaluate, Fields, PointCloud, MeshcatAnimation)

from IPython.display import clear_output

import os
import copy
import time
import numpy as np
from typing import Tuple
import matplotlib.pyplot as plt
from IPython.display import Markdown, display
from dataclasses import dataclass
from dccm_quasistatic.utils.math_utils import (create_square_symmetric_matrix_from_lower_tri_array,
                                               get_n_lower_tri_from_matrix_dim,
                                               matrix_inverse)
from dccm_quasistatic.utils.sample_generator import (SampleGenerator, SampleGeneratorParams)
from dccm_quasistatic.controller.dccm_params import DCCMParams
from dccm_quasistatic.controller_synthesizer.dccm_synthesizer import DCCMSynthesizer
 
from qsim.parser import (
    QuasistaticParser,
    QuasistaticSystemBackend,
    GradientMode,
)

from qsim.simulator import ForwardDynamicsMode, InternalVisualizationType
from qsim.model_paths import models_dir, package_paths_dict

/home/shaoyuan/Documents/Software/quasistatic_simulator/robotics_utilities/qp_derivatives/qp_derivatives.py:2: DrakeDeprecationWarning: Please import from the pydrake.solvers module directly, instead of the deprecated pydrake.solvers.mathematicalprogram submodule. The deprecated code will be removed from Drake on or after 2023-05-01.
  import pydrake.solvers.mathematicalprogram as mp


In [2]:
def synthesize_controller(sample_generator_params: SampleGeneratorParams, dccm_params: DCCMParams):
    file_prefix = "saved_dccm_coeffs/circ_"
    base_filename = f"samples{sample_generator_params.n_samples}_lbw{sample_generator_params.log_barrier_weight}_deg{dccm_params.deg}_beta{str(dccm_params.beta).split('.')[1]}.npy"
    
    # Set up dynamical system
    package_paths_dict["dccm_quasistatic"] =  "/home/shaoyuan/Documents/Software/dccm_quasistatic"
    q_model_path = os.path.join("../../models", "q_sys", "box_pushing.yml")
    q_parser = QuasistaticParser(q_model_path)
    q_sim = q_parser.make_simulator_cpp()
    q_sim_py = q_parser.make_simulator_py(InternalVisualizationType.Cpp)
    
    sample_generator = SampleGenerator(sample_generator_params, q_sim=q_sim, q_sim_py=q_sim_py, parser=q_parser)

    if sample_generator_params.log_barrier_weight == 10:
        samples = sample_generator.generate_circular_traj_samples(b_r = 1.5, s_r_buff = 1.1, visualize=False)
    elif sample_generator_params.log_barrier_weight == 100:
        samples = sample_generator.generate_circular_traj_samples(b_r = 1.1, s_r_buff = 0.22, visualize=False)

    dccm_synth = DCCMSynthesizer(dccm_params)
    success, wijc, lijc = dccm_synth.calculate_dccm_from_samples(*samples)
    if success:
        np.save(f"{file_prefix}wijc_{base_filename}", wijc)
        np.save(f"{file_prefix}lijc_{base_filename}", lijc)
    return success

plt.rcParams.update({'font.size': 40})
linewidth = 5
def draw_plot(t, q, q_des, u, u_des, geodesic, tracking_error):
    clear_output(wait=True)
    fig, axs = plt.subplots(4, 1, figsize=(45, 32))

    axis = axs[0]
    axis.step(t, q, linestyle='solid', linewidth=linewidth)
    axis.step(t, q_des, linestyle='dashed', linewidth=linewidth)
    axis.legend([r'$b_x$', r'$b_y$', r'$b_{\theta}$', r'$s_x$', r'$s_y$',
                 r'$b^*_x$', r'$b^*_y$', r'$b^*_{\theta}$', r'$s^*_x$', r'$s^*_y$'],
                 ncol=2)
    axis.set_ylabel('state')
    axis.set_xlabel('t')

    axis = axs[1]
    axis.step(t, u, linestyle='solid', linewidth=linewidth)
    axis.step(t, u_des, linestyle='dashed', linewidth=linewidth)
    axis.legend([r'$u_{x}$',r'$u_{y}$', r'$u^*_{x}$',r'$u^*_{y}$'],
                ncol=2)
    axis.set_ylabel('u')
    axis.set_xlabel('t')

    axis = axs[2]
    axis.step(t, geodesic, linewidth=linewidth)
    axis.set_ylabel('geodesic length')
    axis.set_xlabel('t')

    axis = axs[3]
    axis.step(t, tracking_error, linewidth=linewidth)
    axis.set_ylabel('tracking error')
    axis.set_xlabel('t')
    plt.show()

# def evaluate_controller(sample_generator_params, dccm_params, traj_id):
#     file_prefix = "saved_dccm_coeffs/circ_"
#     base_filename = f"samples{sample_generator_params.n_samples}_lbw{sample_generator_params.log_barrier_weight}_deg{dccm_params.deg}_beta{str(dccm_params.beta).split('.')[1]}.npy"
    
#     # Set up dynamical system
#     package_paths_dict["dccm_quasistatic"] =  "/home/shaoyuan/Documents/Software/dccm_quasistatic"
#     q_model_path = os.path.join("../../models", "q_sys", "box_pushing.yml")
#     q_parser = QuasistaticParser(q_model_path)
#     q_sim = q_parser.make_simulator_cpp()
#     q_sim_py = q_parser.make_simulator_py(InternalVisualizationType.Cpp)

#     total_duration_s = 10

#     # Load desired trajectory
#     q_desired = np.load(f"saved_dccm_coeffs/circ_traj_{traj_id}.npy")


#     q_sim_py.update_mbp_positions_from_vector(q)
#     q_sim_py.draw_current_configuration()

#     n_steps = int(total_duration_s / sim_p.h)
#     t_log = []
#     q_log = []
#     u_log = []
#     u_desired_log = []
#     geodesic_log = []
#     success_log = []
#     tracking_error_log = []



In [3]:
sample_generator_params = SampleGeneratorParams(
    log_barrier_weight=10,
    n_samples=2000,
    actuated_collision_geomtery_names=["hand::collision"]
)

dccm_params = DCCMParams(
    dim_x=5,
    dim_u=2,
    deg=4,
    beta=0.1,
    n_geodesic_segments=1,
)

betas = [0.5, 0.9]
for beta in betas:
    dccm_params.beta = beta
    success = synthesize_controller(sample_generator_params, dccm_params)
    if success:
        print(f"Successfully synthesized controller with beta = {beta}")
    else:
        print(f"Failed to synthesize controller with beta = {beta}")

sample_generator_params.log_barrier_weight = 100
dccm_params.beta = 0.1
n_samples_list = [100, 500, 1000, 1500, 2000]
n_samples_success = []
for n_samples in n_samples_list:
    sample_generator_params.n_samples = n_samples
    success = synthesize_controller(sample_generator_params, dccm_params)
    n_samples_success.append(success)
    if success:
        print(f"Successfully synthesized controller with n_samples = {n_samples}")
    else:
        print(f"Failed to synthesize controller with n_samples = {n_samples}")

should_continue = False
for i in range(2,5):
    if n_samples_success[i]:
        sample_generator_params.n_samples = n_samples_list[i]
        should_continue = True
if should_continue:
    betas = [0.5, 0.9]
    for beta in betas:
        dccm_params.beta = beta
        success = synthesize_controller(sample_generator_params, dccm_params)
        if success:
            print(f"Successfully synthesized controller with beta = {beta}")
        else:
            print(f"Failed to synthesize controller with beta = {beta}")
else:
    sample_generator_params.log_barrier_weight = 100
    dccm_params.deg = 6
    dccm_params.beta = 0.1
    n_samples_list = [100, 500, 1000, 1500, 2000]
    n_samples_success = []
    for n_samples in n_samples_list:
        sample_generator_params.n_samples = n_samples
        success = synthesize_controller(sample_generator_params, dccm_params)
        n_samples_success.append(success)
        if success:
            print(f"Successfully synthesized controller with n_samples = {n_samples}")
        else:
            print(f"Failed to synthesize controller with n_samples = {n_samples}")

    should_continue = False
    for i in range(2,5):
        if n_samples_success[i]:
            sample_generator_params.n_samples = n_samples_list[i]
            should_continue = True
    if should_continue:
        betas = [0.5, 0.9]
        for beta in betas:
            dccm_params.beta = beta
            success = synthesize_controller(sample_generator_params, dccm_params)
            if success:
                print(f"Successfully synthesized controller with beta = {beta}")
            else:
                print(f"Failed to synthesize controller with beta = {beta}")







Adding constraint for sample  23
